# MssqlDB

Examples with dbatools

## Connect to a SQL Instance

In [ ]:
# -- Connect-DbaInstance 'localhost:8433' as sa
if(-not(Get-InstalledModule -Name dbatools)){
    Install-Module -Name dbatools
}
if(-not(Get-Module -Name dbatools)){
    Import-Module -Name dbatools
}
if([String]::IsNullOrEmpty($sqlcred)){
    $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
    $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred

In [ ]:
# -- Create database
$database = 'tinu'
$dbexists = Get-DbaDatabase -SqlInstance $mssqlsrv  -Database $database
if($dbexists){
    "$database already exists"
}else{
    New-DbaDatabase -SqlInstance $mssqlsrv -Name $database
}

## List all databases

In [ ]:
# -- Get-DbaDatabase
Get-DbaDatabase -SqlInstance $mssqlsrv | FT

## Find database

In [ ]:
# -- Find-DbaDatabase
Find-DbaDatabase -SqlInstance $mssqlsrv -Pattern 'tinu'

## Create database

In [ ]:
# -- Get-DbaDatabase -ExcludeSystem
Get-DbaDatabase -SqlInstance $mssqlsrv -ExcludeSystem

## Max Memory

MaxValue 2147483647 MB use the dynamically memory management. From the total OS memory, reserve 1GB-4GB to the OS itself. For more information see on [Server memory configuration options](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/server-memory-server-configuration-options?view=sql-server-ver15)

In [ ]:
# -- Get-DbaMaxMemory
Get-DbaMaxMemory -SqlInstance $mssqlsrv

In [ ]:
# -- Test-DbaMaxMemory
Test-DbaMaxMemory -SqlInstance $mssqlsrv

In [ ]:
# -- Set-DbaMaxMemory
Set-DbaMaxMemory -SqlInstance $mssqlsrv

## Export JSON

In [ ]:
# -- Export JSON
$DBname     = 'tinu'
$Table      = 'inventory'
$downloads  = Join-Path $env:USERPROFILE -ChildPath downloads
$Backupfile = Join-Path $downloads -ChildPath "$($DBname)-$($Table)-$(Get-Date -f 'yyyy-MM-dd_HH-mm-ss').json"

$SqlQuery   = @"
SELECT * FROM [$DBname].[dbo].[$Table]
GO
"@  

$ResultSqlQuery = Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery
$ResultSqlQuery | ConvertTo-Json -Depth 1 | Set-Content $Backupfile

## Export CSV

In [ ]:
# -- Export CSV
$DBname     = 'tinu'
$Table      = 'inventory'
$downloads  = Join-Path $env:USERPROFILE -ChildPath downloads
$Backupfile = Join-Path $downloads -ChildPath "$($DBname)-$($Table)-$(Get-Date -f 'yyyy-MM-dd_HH-mm-ss').csv"

$SqlQuery   = @"
SELECT * FROM [$DBname].[dbo].[$Table]
GO
"@  

$ResultSqlQuery = Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery
$ResultSqlQuery | ConvertTo-Csv -Delimiter ';' -NoTypeInformation | Set-Content $Backupfile

## Import JSON

Performs a bulk insert of all the data in MongoDB.local.tinu.inventory.json. If the table inventory does not exist, it will be created with inefficient but forgiving DataTypes.

In [ ]:
# -- Import JSON
$DBname     = 'tinu'
$Table      = 'inventory'
$downloads  = Join-Path $env:HOME -ChildPath downloads
$json_file  = Join-Path $downloads -ChildPath 'MongoDB.local.tinu.inventory.json'
$columns = @(
    'RunTime','DeviceType','DeviceName','Physical cores','MemoryGB','DiskspaceGB','PurchaseDate','Price','Payment','Warranty','Link'
)
$dataset = Get-Content $json_file | ConvertFrom-Json | Select-Object $columns | ConvertTo-DbaDataTable
$dataset | Write-DbaDbTableData -SqlInstance $mssqlsrv -Database $DBname -Table $table -AutoCreateTable -KeepNulls #-Truncate

## Create Primary Key Integer

In [ ]:
# -- Create Primary Key as Interger
$DBname     = 'tinu'
$Table      = 'inventory'
$SqlQuery   = @"
ALTER TABLE [$DBname].[dbo].[$Table]
    ADD ID INT IDENTITY (1,1) NOT NULL PRIMARY KEY
GO
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery

## Create Primary Key GUID

In [ ]:
# -- Create Primary Key as GUID
$DBname     = 'tinu'
$Table      = 'inventory'
$SqlQuery   = @"
ALTER TABLE [$DBname].[dbo].[$Table]
    ADD ID UNIQUEIDENTIFIER NOT NULL DEFAULT NEWSEQUENTIALID() PRIMARY KEY
GO
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery

## Drop Table

In [ ]:
# -- Drop Table
$DBname     = 'tinu'
$Table      = 'inventory'
$SqlQuery   = @"
DROP TABLE [$DBname].[dbo].[$Table]
GO
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery